# Market demand prediction
*This notebook illustrates methods to predict the demand series of a supply chain.*  
*Use the virtual environment logproj.yml to run this notebook.*
***
Alessandro Tufano 2020

### Import packages

In [1]:
# %% append functions path
import sys; sys.path.insert(0, '..') #add the above level with the package

from IPython.display import display, HTML #display dataframe

#import utilities
from logproj.utilities import creaCartella

### Set data fields

In [2]:
string_casestudy = 'TOY_DATA'

### Import data

In [3]:
# %% import data
from logproj.data_generator_distribution import generateDistributionData

#random generation of distribution data
_, _, _, D_mov = generateDistributionData(num_movements=500)


In [4]:
#print nodes dataframe
display(HTML(D_mov.to_html()))

,LOADING_NODE,LOADING_NODE_LATITUDE,LOADING_NODE_LONGITUDE,PTA_FROM,PTD_FROM,ATA_FROM,ATD_FROM,DISCHARGING_NODE,DISCHARGING_LATITUDE,DISCHARGING_LONGITUDE,PTA_TO,PTD_TO,ATA_TO,ATD_TO,ITEMCODE,PRODUCT_FAMILY,CLIENT,VEHICLE_CODE,VOYAGE_CODE,QUANTITY,TIMESTAMP_IN,PACKAGE_DESCRIPTION
0,15,41.540530,14.724962,2020-01-02 00:00:00.000000,2020-01-02 01:00:00.000000,2020-01-01 23:41:41.840378,2020-01-02 01:00:17.354662,1,41.523632,14.996929,2020-01-02 07:53:33.798680,2020-01-02 08:53:33.798680,2020-01-02 07:56:10.108479,2020-01-02 09:18:10.026347,1,PRODUCT_FAMILY 2,CLIENT 2,TRUCK 1,0,9.901046,2019-12-28 04:47:46.360471,TEU CONTAINER
0,8,41.554176,14.985382,2020-01-02 10:36:13.328101,2020-01-02 11:36:13.328101,2020-01-02 10:21:24.572817,2020-01-02 11:21:18.344120,19,41.776415,14.253812,2020-01-03 10:49:35.681406,2020-01-03 11:49:35.681406,2020-01-03 10:44:15.101932,2020-01-03 11:57:39.216534,0,PRODUCT_FAMILY 2,CLIENT 1,TRUCK 1,0,1.137379,2019-12-31 17:12:10.653280,TEU CONTAINER
0,22,41.833904,15.025307,2020-01-03 13:21:31.280460,2020-01-03 14:21:31.280460,2020-01-03 13:06:32.223421,2020-01-03 14:03:43.006742,18,41.590095,15.056044,2020-01-03 22:09:21.287581,2020-01-03 23:09:21.287581,2020-01-03 22:10:39.589533,2020-01-03 23:16:48.063738,1,PRODUCT_FAMILY 2,CLIENT 2,TRUCK 1,0,8.838956,2019-12-30 17:42:56.287196,FEU CONTAINER
0,6,41.580115,14.535136,2020-01-04 00:58:49.389328,2020-01-04 01:58:49.389328,2020-01-04 00:42:09.518775,2020-01-04 02:34:33.551500,8,41.554176,14.985382,2020-01-04 12:03:17.262654,2020-01-04 13:03:17.262654,2020-01-04 12:00:41.877276,2020-01-04 13:12:25.432411,1,PRODUCT_FAMILY 2,CLIENT 1,TRUCK 1,0,3.343371,2020-01-02 18:42:34.442798,TEU CONTAINER
0,22,41.833904,15.025307,2020-01-04 13:17:58.101967,2020-01-04 14:17:58.101967,2020-01-04 13:35:21.310406,2020-01-04 14:32:56.357428,12,41.434154,14.419739,2020-01-05 21:57:48.266055,2020-01-05 22:57:48.266055,2020-01-05 21:54:15.231765,2020-01-05 22:56:27.157034,0,PRODUCT_FAMILY 2,CLIENT 1,TRUCK 1,0,5.163902,2020-01-01 12:29:31.169930,TEU CONTAINER
0,14,41.600354,14.300657,2020-01-05 23:57:57.103236,2020-01-06 00:57:57.103236,2020-01-06 00:11:43.596909,2020-01-06 01:02:06.113977,18,41.590095,15.056044,2020-01-07 01:27:04.978542,2020-01-07 02:27:04.978542,2020-01-07 01:44:18.009787,2020-01-07 02:55:31.020177,1,PRODUCT_FAMILY 2,CLIENT 1,TRUCK 1,0,9.634099,2020-01-01 11:28:07.988926,TEU CONTAINER
0,19,41.776415,14.253812,2020-01-07 04:57:06.112180,2020-01-07 05:57:06.112180,2020-01-07 04:45:00.963420,2020-01-07 06:16:19.837929,17,41.942656,14.555851,2020-01-07 19:00:15.316606,2020-01-07 20:00:15.316606,2020-01-07 18:59:56.075481,2020-01-07 19:56:40.227549,0,PRODUCT_FAMILY 2,CLIENT 1,TRUCK 1,0,6.913050,2019-12-22 17:21:24.968251,TEU CONTAINER
0,23,41.620753,14.996953,2020-01-07 21:01:15.439307,2020-01-07 22:01:15.439307,2020-01-07 21:14:08.731590,2020-01-07 21:54:52.021509,14,41.600354,14.300657,2020-01-08 14:06:31.032634,2020-01-08 15:06:31.032634,2020-01-08 13:57:12.263579,2020-01-08 15:31:55.141246,1,PRODUCT_FAMILY 2,CLIENT 1,TRUCK 1,0,6.063187,2019-12-29 04:46:21.219637,TEU CONTAINER
0,22,41.833904,15.025307,2020-01-08 16:30:34.314648,2020-01-08 17:30:34.314648,2020-01-08 16:38:07.826741,2020-01-08 17:20:30.145055,22,41.833904,15.025307,2020-01-08 17:30:34.314648,2020-01-08 18:30:34.314648,2020-01-08 18:05:15.464281,2020-01-08 18:23:03.978275,1,PRODUCT_FAMILY 2,CLIENT 2,TRUCK 1,0,2.001165,2019-12-25 19:41:32.094700,FEU CONTAINER
0,22,41.833904,15.025307,2020-01-08 21:47:31.944667,2020-01-08 22:47:31.944667,2020-01-08 21:16:57.870718,2020-01-08 22:39:27.861249,22,41.833904,15.025307,2020-01-08 22:47:31.944667,2020-01-08 23:47:31.944667,2020-01-08 22:42:55.661619,2020-01-08 23:53:52.696060,1,PRODUCT_FAMILY 2,CLIENT 1,TRUCK 1,0,2.646473,2020-01-01 20:40:12.831591,FEU CONTAINER


### Create folder hierarchy

In [5]:
# %% create folder hierarchy
pathResults = 'C:\\Users\\aletu\\desktop'
_, root_path = creaCartella(pathResults,f"{string_casestudy}_results")
_, prediction_results_path = creaCartella(root_path,f"P9_predictions")

Cartella TOY_DATA_results già esistente
Cartella P9_predictions già esistente


### Build the time series

In [6]:
D_time = D_mov.groupby(['TIMESTAMP_IN']).agg({'QUANTITY':['sum'],'TIMESTAMP_IN':['size']}).reset_index()
D_time.columns = ['TIMESTAMP', 'QUANTITY','COUNT']
#print the series
display(HTML(D_time.to_html()))

,TIMESTAMP,QUANTITY,COUNT
0,2019-12-22 17:21:24.968251,6.913050,1
1,2019-12-23 20:43:58.941712,9.205869,1
2,2019-12-25 19:41:32.094700,2.001165,1
3,2019-12-27 21:36:55.200839,2.312064,1
4,2019-12-28 04:47:46.360471,9.901046,1
5,2019-12-29 01:16:13.509462,5.496678,1
6,2019-12-29 04:46:21.219637,6.063187,1
7,2019-12-29 22:51:33.036547,8.893071,1
8,2019-12-30 17:42:56.287196,8.838956,1
9,2019-12-31 17:12:10.653280,1.137379,1


### SARIMA predictions

In [7]:
import pandas as pd
import datetime as date
import numpy as np
from logproj.P9workloadPrediction.demandPredicter import predictWithARIMA
def PROD_PREDICT_SARIMA(D_time, qtyVariable, countVariable,prediction_results_path,filterVariable=[], samplingInterval='week'):
    
    def nestedPredictionSARIMA(D_results, ss, nomecartella):
        #create folder with results
        _, current_dir_results = creaCartella(prediction_results_path,f"{str(nomecartella)}")
            
        print(f"***********{ss}*************")
        if len(ss)>0:
            D_series=D_time[D_time[filterVariable]==ss]
        else:
            D_series=D_time
        
        #save report txt
        file = open(f"{current_dir_results}\\{reportFilename}.txt", "w") 
        file.write(f"{str(date.datetime.now())} STARTING PREDICTIONS \r") 
        file.close() 
        
        # set initial
        tot_qty = np.nansum(D_series[qtyVariable])
        tot_lns = np.nansum(D_series[countVariable])
        QTY_STATIONARY_TRANSFORM=''
        QTY_SARIMA_MODEL=''
        QTY_SARIMA_ACCURACY = ''
        LNS_STATIONARY_TRANSFORM=''
        LNS_SARIMA_MODEL=''
        LNS_SARIMA_ACCURACY = ''
        
        if len(D_series)>=12: #I need at least 12 points (e.g. 1 year expressed in  months)
            
            # predict quantities
            stationary_model, fig_CF, figure_forecast, figure_residuals, resultModel = predictWithARIMA(D_series, seriesVariable=qtyVariable, samplingInterval=samplingInterval, signifAlpha=0.05,maxValuesSelected=2)
            
            if len(stationary_model)>0:
                #save figures
                fig_CF.get_figure().savefig(f"{current_dir_results}\\{ss}_quantities_CF.png")
                figure_forecast.savefig(f"{current_dir_results}\\{ss}_quantities_ARIMA_forecast.png")
                figure_residuals.savefig(f"{current_dir_results}\\{ss}_quantities_ARIMA_residuals.png")
                plt.close('all')
            
                #save params
                QTY_STATIONARY_TRANSFORM=stationary_model
                QTY_SARIMA_MODEL=str({'p':resultModel['p'],'d':resultModel['d'],'q':resultModel['q']})
                QTY_SARIMA_ACCURACY = resultModel['aic']
                
                #save report txt
                file = open(f"{current_dir_results}\\{reportFilename}.txt", "a") 
                file.write(f"{str(date.datetime.now())} quantities: Predictions built\r") 
                file.close() 
            else:
                #save report txt
                file = open(f"{current_dir_results}\\{reportFilename}.txt", "a") 
                file.write(f"{str(date.datetime.now())} quantities: no stationary series, no ARIMA Predictions built\r") 
                file.close() 
                
            
            #predict lines
            stationary_model, fig_CF, figure_forecast, figure_residuals, resultModel = predictWithARIMA(D_series, seriesVariable=countVariable, samplingInterval=samplingInterval, signifAlpha=0.05,maxValuesSelected=2)
            
            if len(stationary_model) >0: # se le predizioni sono riuscite
                #save figures
                fig_CF.get_figure().savefig(f"{current_dir_results}\\{ss}_lines_CF.png")
                figure_forecast.savefig(f"{current_dir_results}\\{ss}_lines_ARIMA_forecast.png")
                figure_residuals.savefig(f"{current_dir_results}\\{ss}_lines_ARIMA_residuals.png")
                plt.close('all')
            
                LNS_STATIONARY_TRANSFORM=stationary_model
                LNS_SARIMA_MODEL=str({'p':resultModel['p'],'d':resultModel['d'],'q':resultModel['q']})
                LNS_SARIMA_ACCURACY = resultModel['aic']
            
            
                #save report txt
                file = open(f"{current_dir_results}\\{reportFilename}.txt", "a") 
                file.write(f"{str(date.datetime.now())} lines: Predictions built\r") 
                file.close() 
            else:
                #save report txt
                file = open(f"{current_dir_results}\\{reportFilename}.txt", "a") 
                file.write(f"{str(date.datetime.now())} lines: no stationary series, no ARIMA Predictions built\r") 
                file.close() 
            
            
        
        
        else:
            
            
            #save report txt
            file = open(f"{current_dir_results}\\{reportFilename}.txt", "w") 
            file.write(f"{str(date.datetime.now())} Not ehough input points to build a time series\r") 
            file.close() 
            
            
        # append results to dataframe
        D_results=D_results.append(pd.DataFrame([[ss, tot_qty, tot_lns, QTY_STATIONARY_TRANSFORM, QTY_SARIMA_MODEL,
                                                QTY_SARIMA_ACCURACY, LNS_STATIONARY_TRANSFORM, LNS_SARIMA_MODEL,
                                                LNS_SARIMA_ACCURACY]], columns = D_results.columns))
        return D_results
    
    
    
    
    
    reportFilename='report_SARIMA'
    
    resultscolumn=['SERVICETYPE','QUANTITIES','LINES','QTY_STATIONARY_TRANSFORM','QTY_SARIMA_MODEL','QTY_SARIMA_ACCURACY',
               'LNS_STATIONARY_TRANSFORM','LNS_SARIMA_MODEL','LNS_SARIMA_ACCURACY']
    
    D_results=pd.DataFrame(columns=resultscolumn)
    print("CIAOOOO")
    print(D_results)
    # genero i trend globali
    D_results = nestedPredictionSARIMA(D_results,[], nomecartella='globalResults')
    
    if len(filterVariable)>0:
        #itero sulle famiglie di prodotto
        st = list(set(D_time[filterVariable]))
        
        for ss in st:
            #ss='zz'
            try:
                D_results = nestedPredictionSARIMA(D_results, ss, nomecartella=ss)
            except Exception as e:
                print(f"*=*=*=*=*=*=ERROR*=*=*= {e}")
        #ss='zz'
        
    
            
            
            
    #  SAVE dataframe results
    D_results.to_excel(f"{prediction_results_path}\\pred_results_SARIMA.xlsx")
    return True

In [8]:
PROD_PREDICT_SARIMA(D_time, 
                    qtyVariable='QUANTITY', 
                    countVariable='COUNT',
                    prediction_results_path = prediction_results_path, 
                    filterVariable = [],
                    samplingInterval='week')

CIAOOOO
Empty DataFrame
Columns: [SERVICETYPE, QUANTITIES, LINES, QTY_STATIONARY_TRANSFORM, QTY_SARIMA_MODEL, QTY_SARIMA_ACCURACY, LNS_STATIONARY_TRANSFORM, LNS_SARIMA_MODEL, LNS_SARIMA_ACCURACY]
Index: []
Cartella globalResults già esistente
***********[]*************


KeyError: "['TIMESTAMP_IN'] not in index"